In [7]:
from google.colab import drive
import os
import gdown # Library to download files from Google Drive
!gdown 1JdGaXmUnItfCn8ucn_5kBfDeh3NzrmMs # Google Drive ID of the zip file to be downloaded


Downloading...
From (original): https://drive.google.com/uc?id=1JdGaXmUnItfCn8ucn_5kBfDeh3NzrmMs
From (redirected): https://drive.google.com/uc?id=1JdGaXmUnItfCn8ucn_5kBfDeh3NzrmMs&confirm=t&uuid=c941ca56-22bd-493f-877e-e4cceeb1bc9d
To: /content/images.zip adlı dosyanın kopyası
100% 128M/128M [00:00<00:00, 216MB/s]
unzip:  cannot find or open images.zip, images.zip.zip or images.zip.ZIP.


In [14]:
!unzip -oq images # Unzip the file downloaded. Options -o and -q overwrites the files if exists already and disables printing out the extracted files, respectively.

In [16]:

root_dir = '/content/images'


In [21]:
from PIL import Image
import os

root_dir = '/content/images'

# Function to calculate average resolution of images in a directory
def calculate_average_resolution(directory):
    total_width = 0
    total_height = 0
    num_images = 0

    # Loop through each file in the directory
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        # Check if the file is an image
        if os.path.isfile(filepath) and any(filename.lower().endswith(ext) for ext in ['.png', '.jpg', '.jpeg']):
            # Open the image and get its dimensions
            with Image.open(filepath) as img:
                width, height = img.size
                total_width += width
                total_height += height
                num_images += 1

    if num_images == 0:
        print("No images found in the directory.")
        return None
    else:
        avg_width = total_width // num_images
        avg_height = total_height // num_images
        return (avg_width, avg_height, num_images)

# Calculate average resolution and total number of images
result = calculate_average_resolution(root_dir)

if result:
    avg_resolution, num_images = result[:2], result[2]
    print(f"Total number of images: {num_images}")
    print(f"Average resolution: {avg_resolution[0]}x{avg_resolution[1]} pixels")
else:
    print("Unable to calculate average resolution.")


Total number of images: 636
Average resolution: 1194x867 pixels


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate

# Image input branch
image_input = Input(shape=(224, 224, 3))
x = Conv2D(32, (3, 3), activation='relu')(image_input)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)

# Font type input branch
font_input = Input(shape=(num_font_types,))
# Color palette input branch
color_input = Input(shape=(num_colors,))

# Concatenate all features
concatenated = Concatenate()([x, font_input, color_input])

# Dense layers for prediction
y = Dense(128, activation='relu')(concatenated)
y = Dense(64, activation='relu')(y)
output = Dense(1)(y)  # CPI output

model = Model(inputs=[image_input, font_input, color_input], outputs=output)
model.compile(optimizer='adam', loss='mean_absolute_error')

model.summary()